# Project EDA

In [1]:
# Setup - Run only once per Kernel App
%conda install https://anaconda.org/conda-forge/openjdk/11.0.1/download/linux-64/openjdk-11.0.1-hacce0ff_1021.tar.bz2

# install PySpark
%pip install pyspark==3.4.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Retrieving notices: ...working... done



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - conda-forge/openjdk/11.0.1/download/linux-64::openjdk==11.0.1=hacce0ff_1021


The following NEW packages will be INSTALLED:

  openjdk            conda-forge/openjdk/11.0.1/download/linux-64::openjdk-11.0.1-hacce0ff_1021 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.
  Using cached pyspark-3.4.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sagemaker-user/.ivy2/cache
The jars for the packages stored in: /home/sagemaker-user/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f4dce436-7a93-4c7a-9667-4fb4cc922d58;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 387ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------

3.4.0


In [3]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_comments = "project/comments/yyyy=*"
output_prefix_data_submissions = "project/submissions/yyyy=*"
s3_path_comments = f"s3a://{bucket}/{output_prefix_data_comments}"
s3_path_submissions = f"s3a://{bucket}/{output_prefix_data_submissions}"

print(f"reading comments from {s3_path_comments}")
comments = spark.read.parquet(s3_path_comments, header=True)
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}")

print(f"reading submissions from {s3_path_submissions}")
submissions = spark.read.parquet(s3_path_submissions, header=True)
print(f"shape of the submissions dataframe is {submissions.count():,}x{len(submissions.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
reading comments from s3a://sagemaker-us-east-1-739730413582/project/comments/yyyy=*


24/11/12 02:51:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


shape of the comments dataframe is 15,079,731x17
reading submissions from s3a://sagemaker-us-east-1-739730413582/project/submissions/yyyy=*


shape of the submissions dataframe is 330,987x21
CPU times: user 2.96 s, sys: 336 ms, total: 3.3 s
Wall time: 10min 35s


# Comment Data EDA

In [4]:
comments.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: double (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



In [7]:
submissions.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- edited: double (nullable = true)
 |-- id: string (nullable = true)
 |-- is_self: boolean (nullable = true)
 |-- locked: boolean (nullable = true)
 |-- num_comments: long (nullable = true)
 |-- over_18: boolean (nullable = true)
 |-- quarantine: boolean (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- selftext: string (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)



In [16]:
from pyspark.sql.functions import avg, count, length, size, split

comments_df = comments.groupBy('subreddit').agg(
    avg("controversiality").alias("avg_controversiality"),
    avg("score").alias("avg_score"),
    avg("gilded").alias("avg_gilded"),
    count("*").alias("num_comments"),
    avg(length("body")).alias("avg_body_length"),
    avg(size(split("body", " "))).alias("avg_word_count")
).collect()

print(comments_df.show(20))

AttributeError: 'list' object has no attribute 'show'

In [20]:
import pandas as pd 
comments_pandas_df = pd.DataFrame([row.asDict() for row in comments_df])
comments_pandas_df

,subreddit,avg_controversiality,avg_score,avg_gilded,num_comments,avg_body_length,avg_word_count
0,Republican,0.081086,3.538371,0.000000,116925,214.154227,33.296583
1,GreenParty,0.014804,2.232420,0.000000,2702,356.502961,59.686158
2,politics,0.032764,10.777683,0.000058,11541659,176.805559,30.023606
3,Libertarian,0.055770,5.314409,0.000016,364029,227.205272,38.652701
4,PoliticalDiscussion,0.045728,3.779797,0.000016,565016,342.471776,56.075037
5,Ask_Politics,0.010950,1.989933,0.000000,5662,614.187919,91.080890
6,democrats,0.015823,6.559963,0.000012,322764,146.520991,24.843443
7,Conservative,0.049312,7.352351,0.000040,2160974,98.997078,17.005424


In [21]:
comments_pandas_df.to_csv('subreddit_comment_agg.csv')

In [22]:
submissions_df = submissions.groupBy('subreddit').agg(
    avg("num_comments").alias("avg_num_comments"),
    avg("score").alias("avg_score"),
    count("*").alias("num_comments"),
    avg(length("selftext")).alias("avg_selftext_length"),
    avg(size(split("selftext", " "))).alias("avg_word_count_selftext"),
    avg(length("title")).alias("avg_title_length"),
    avg(size(split("title", " "))).alias("avg_word_count_title")
)

print(submissions_df.show(10))

+-------------------+------------------+------------------+------------+-------------------+-----------------------+-----------------+--------------------+
|          subreddit|  avg_num_comments|         avg_score|num_comments|avg_selftext_length|avg_word_count_selftext| avg_title_length|avg_word_count_title|
+-------------------+------------------+------------------+------------+-------------------+-----------------------+-----------------+--------------------+
|         Republican|  4.67120218579235|20.054426229508195|       18300| 18.366120218579233|     3.5198907103825134|71.72273224043715|   11.53743169398907|
|         GreenParty|1.7677053824362605| 6.259678942398489|        1059| 162.91501416430594|     25.302171860245515|89.45325779036827|  14.349386213408875|
|           politics| 55.03217459301912|490.96754105352954|      140670|  5.183016990118718|     1.4054240420843107|  81.381644984716|  13.013300632686429|
|        Libertarian|18.058145223286044| 67.99881193654343|     

In [23]:
submissions_df.toPandas().to_csv('subreddit_submissions_counts.csv')

# Local EDA

In [3]:
import pyarrow
import pandas as pd

comments = pd.read_parquet('comments.parquet')
submissions = pd.read_parquet('submissions.parquet')

In [4]:
comments.columns

Index(['author', 'author_flair_css_class', 'author_flair_text', 'body',
       'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded',
       'id', 'link_id', 'parent_id', 'retrieved_on', 'score', 'stickied',
       'subreddit', 'subreddit_id'],
      dtype='object')

In [5]:
comments['body_char_count'] = comments['body'].apply(len)
comments['body_word_count'] = comments['body'].apply(lambda x: len(x.split()))
comments.groupby('subreddit').agg(
    author_count=('author', 'count'),
    author_flair_text_count=('author_flair_text', 'count'),
    avg_character_count=('body_char_count', 'mean'),
    med_character_count=('body_char_count', 'median'),
    avg_word_count=('body_word_count', 'mean'),
    med_word_count=('body_word_count', 'median'),
    avg_controversiality=('controversiality', 'mean'),
    med_controversiality=('controversiality', 'median'),
    avg_score=('score', 'mean'),
    med_score=('score', 'median'),
    post_count=('subreddit', 'size')
)

,author_count,author_flair_text_count,avg_character_count,med_character_count,avg_word_count,med_word_count,avg_controversiality,med_controversiality,avg_score,med_score,post_count
subreddit,,,,,,,,,,,
Conservative,2160974,576717,98.997078,31.0,17.010845,6.0,0.049312,0.0,7.352351,1.0,2160974
Liberal,73665,14,177.903306,86.0,30.601290,15.0,0.030150,0.0,4.004846,1.0,73665
Libertarian,364029,59771,227.205272,121.0,38.796008,21.0,0.055770,0.0,5.314409,1.0,364029
Republican,116925,6700,214.154227,93.0,33.330468,17.0,0.081086,0.0,3.538371,1.0,116925
democrats,322764,1360,146.520991,72.0,24.874153,13.0,0.015823,0.0,6.559963,1.0,322764


In [6]:
submissions.columns

Index(['author', 'author_flair_css_class', 'author_flair_text', 'created_utc',
       'distinguished', 'domain', 'edited', 'id', 'is_self', 'locked',
       'num_comments', 'over_18', 'quarantine', 'retrieved_on', 'score',
       'selftext', 'stickied', 'subreddit', 'subreddit_id', 'title', 'url'],
      dtype='object')

In [7]:
submissions['title_char_count'] = submissions['title'].apply(len)
submissions['title_word_count'] = submissions['title'].apply(lambda x: len(x.split()))
submissions['selftext_char_count'] = submissions['selftext'].apply(len)
submissions['selftext_word_count'] = submissions['selftext'].apply(lambda x: len(x.split()))

submissions.groupby('subreddit').agg(
    author_count=('author', 'count'),
    author_flair_text_count=('author_flair_text', 'count'),
    title_avg_character_count=('title_char_count', 'mean'),
    title_med_character_count=('title_char_count', 'median'),
    title_avg_word_count=('title_word_count', 'mean'),
    title_med_word_count=('title_word_count', 'median'),
    selftext_avg_character_count=('selftext_char_count', 'mean'),
    selftexxt_med_character_count=('selftext_char_count', 'median'),
    selftext_avg_word_count=('selftext_word_count', 'mean'),
    selftext_med_word_count=('selftext_word_count', 'median'),
    avg_score=('score', 'mean'),
    med_score=('score', 'median'),
    post_count=('subreddit', 'size'),
    num_comments=('num_comments', 'sum')
)

,author_count,author_flair_text_count,title_avg_character_count,title_med_character_count,title_avg_word_count,title_med_word_count,selftext_avg_character_count,selftexxt_med_character_count,selftext_avg_word_count,selftext_med_word_count,avg_score,med_score,post_count,num_comments
subreddit,,,,,,,,,,,,,,
Ask_Politics,1887,16,74.588765,64.0,12.517753,11.0,48.917329,9.0,7.763646,1.0,1.810281,1.0,1887,3793
Conservative,113148,47766,71.987645,68.0,11.646339,11.0,14.643290,0.0,2.213676,0.0,50.319060,1.0,113148,1645153
Liberal,4537,3,67.264051,62.0,11.170377,10.0,145.754243,9.0,23.920212,1.0,25.658144,1.0,4537,55737
Libertarian,14309,5317,62.771333,52.0,10.318191,8.0,181.889650,0.0,29.727933,0.0,67.998812,1.0,14309,258394
PoliticalDiscussion,18325,19,73.049550,62.0,12.361364,10.0,113.099372,9.0,17.533861,1.0,9.670505,1.0,18325,342636
Republican,18300,1139,71.722732,69.0,11.495355,11.0,18.366120,0.0,2.786612,0.0,20.054426,1.0,18300,85483
democrats,23289,262,74.171970,70.0,12.103826,11.0,46.634978,0.0,7.566748,0.0,82.163296,1.0,23289,261452
politics,140670,29445,81.381645,73.0,12.989856,12.0,5.183017,0.0,0.547373,0.0,490.967541,1.0,140670,7741376


In [8]:
submissions['datetime'] = pd.to_datetime(submissions['created_utc'], unit='s')
submissions['datetime'].dt.month
comments['datetime'] = pd.to_datetime(comments['created_utc'], unit='s')

In [9]:
submissions_over_time = submissions.groupby(['subreddit', submissions['datetime'].dt.month, submissions['datetime'].dt.year]).agg(
    author_count=('author', 'count'),
    author_flair_text_count=('author_flair_text', 'count'),
    title_avg_character_count=('title_char_count', 'mean'),
    title_med_character_count=('title_char_count', 'median'),
    title_avg_word_count=('title_word_count', 'mean'),
    title_med_word_count=('title_word_count', 'median'),
    selftext_avg_character_count=('selftext_char_count', 'mean'),
    selftexxt_med_character_count=('selftext_char_count', 'median'),
    selftext_avg_word_count=('selftext_word_count', 'mean'),
    selftext_med_word_count=('selftext_word_count', 'median'),
    avg_score=('score', 'mean'),
    med_score=('score', 'median'),
    post_count=('subreddit', 'size'),
    num_comments=('num_comments', 'sum')
)
submissions_over_time

author_count  author_flair_text_count  \
subreddit    datetime datetime                                          
Ask_Politics 1        2024                87                        0   
             2        2024               103                        0   
             3        2024               111                        0   
             4        2024                91                        0   
             5        2024                96                        4   
...                                      ...                      ...   
politics     8        2023              8976                     2208   
             9        2023              9677                     1803   
             10       2023              9948                     1971   
             11       2023              8141                     1747   
             12       2023              7785                     1875   

                                title_avg_character_count  \
subreddit    datetime datetime                              
Ask_Politics 1        2024                      81.057471   
             2        2024                      77.097087   
             3        2024                      71.027027   
             4        2024                      89.890110   
             5        2024                      70.760417   
...                                                   ...   
politics     8        2023                      79.944296   
             9        2023                      81.602563   
             10       2023                     100.324889   
             11       2023                      87.470950   
             12       2023                      79.404496   

                                title_med_character_count  \
subreddit    datetime datetime                              
Ask_Politics 1        2024                           67.0   
             2        2024                           70.0   
             3        2024                           58.0   
             4        2024                           83.0   
             5        2024                           60.5   
...                                                   ...   
politics     8        2023                           74.0   
             9        2023                           73.0   
             10       2023                           77.0   
             11       2023                           75.0   
             12       2023                           74.0   

                                title_avg_word_count  title_med_word_count  \
subreddit    datetime datetime                                               
Ask_Politics 1        2024                 13.931034                  12.0   
             2        2024                 13.029126                  12.0   
             3        2024                 11.783784                  10.0   
             4        2024                 14.901099                  13.0   
             5        2024                 11.635417                  10.0   
...                                              ...                   ...   
politics     8        2023                 12.699643                  12.0   
             9        2023                 12.906479                  12.0   
             10       2023                 15.225472                  12.0   
             11       2023                 13.657536                  12.0   
             12       2023                 12.651124                  12.0   

                                selftext_avg_character_count  \
subreddit    datetime datetime                                 
Ask_Politics 1        2024                          9.000000   
             2        2024                          8.912621   
             3        2024                          8.918919   
             4        2024                          8.604396   
             5        2024                         44.885417   
...                            

In [10]:
comments

comments_over_time = comments.groupby(['subreddit', comments['datetime'].dt.month, comments['datetime'].dt.year]).agg(
    author_count=('author', 'count'),
    author_flair_text_count=('author_flair_text', 'count'),
    avg_character_count=('body_char_count', 'mean'),
    med_character_count=('body_char_count', 'median'),
    avg_word_count=('body_word_count', 'mean'),
    med_word_count=('body_word_count', 'median'),
    avg_controversiality=('controversiality', 'mean'),
    med_controversiality=('controversiality', 'median'),
    avg_score=('score', 'mean'),
    med_score=('score', 'median'),
    post_count=('subreddit', 'size')
)

In [23]:
comments_over_time.to_csv('comments_subreddit_time.csv')
submissions_over_time.to_csv('submissions_subreddit_time.csv')